In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.impute import KNNImputer , SimpleImputer
from sklearn.preprocessing import OneHotEncoder , MinMaxScaler , StandardScaler, RobustScaler 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate 
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from category_encoders import BinaryEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

In [149]:
df = pd.read_excel("Data_Train.xlsx")
df

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [151]:
df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [152]:
df.dropna(inplace=True)

In [153]:
df.duplicated().sum()

220

In [154]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)

In [155]:
sorted(df['Airline'].unique())

['Air Asia',
 'Air India',
 'GoAir',
 'IndiGo',
 'Jet Airways',
 'Jet Airways Business',
 'Multiple carriers',
 'Multiple carriers Premium economy',
 'SpiceJet',
 'Trujet',
 'Vistara',
 'Vistara Premium economy']

In [156]:
df['Airline'].value_counts()

Jet Airways                          3700
IndiGo                               2043
Air India                            1694
Multiple carriers                    1196
SpiceJet                              815
Vistara                               478
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: Airline, dtype: int64

I will drop Trujet because it appears only once in the DataFrame.

In [157]:
droped = df[df["Airline"] == "Trujet"].index
df.drop(droped, inplace=True)

In [158]:
df["Date_of_Journey"].unique()

array(['24/03/2019', '1/05/2019', '9/06/2019', '12/05/2019', '01/03/2019',
       '24/06/2019', '12/03/2019', '27/05/2019', '1/06/2019',
       '18/04/2019', '9/05/2019', '24/04/2019', '3/03/2019', '15/04/2019',
       '12/06/2019', '6/03/2019', '21/03/2019', '3/04/2019', '6/05/2019',
       '15/05/2019', '18/06/2019', '15/06/2019', '6/04/2019',
       '18/05/2019', '27/06/2019', '21/05/2019', '06/03/2019',
       '3/06/2019', '15/03/2019', '3/05/2019', '9/03/2019', '6/06/2019',
       '24/05/2019', '09/03/2019', '1/04/2019', '21/04/2019',
       '21/06/2019', '27/03/2019', '18/03/2019', '12/04/2019',
       '9/04/2019', '1/03/2019', '03/03/2019', '27/04/2019'], dtype=object)

I will perform some feature engineering, extracting the day and month, and then drop this column.

In [159]:
df["Date_of_Journey"] = pd.to_datetime(df["Date_of_Journey"] , format="%d/%m/%Y",errors = "coerce")

In [160]:
df["Day_of_Journey"] = df["Date_of_Journey"].dt.day_name()
df["Month_of_Journey"] = df["Date_of_Journey"].dt.month_name()
df.drop('Date_of_Journey',axis=1,inplace=True)

In [161]:
df.duplicated().sum()

579

In [162]:
df["Month_of_Journey"].value_counts()

May      3395
June     3311
March    2677
April    1078
Name: Month_of_Journey, dtype: int64

In [163]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)

In [164]:
df['Source'].unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [165]:
df['Source'].value_counts()

Delhi       4197
Kolkata     2752
Banglore    1986
Mumbai       628
Chennai      319
Name: Source, dtype: int64

In [166]:
df['Destination'].unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [167]:
df['Destination'].value_counts()

Cochin       4197
Banglore     2752
Delhi        1073
New Delhi     913
Hyderabad     628
Kolkata       319
Name: Destination, dtype: int64

In [168]:
df['Dep_Time'].unique()

array(['22:20', '05:50', '09:25', '18:05', '16:50', '09:00', '18:55',
       '08:00', '08:55', '11:25', '09:45', '20:20', '11:40', '21:10',
       '17:15', '16:40', '08:45', '14:00', '20:15', '16:00', '14:10',
       '22:00', '04:00', '21:25', '21:50', '07:00', '07:05', '09:50',
       '14:35', '10:35', '15:05', '14:15', '06:45', '20:55', '11:10',
       '05:45', '19:00', '23:05', '11:00', '09:35', '21:15', '23:55',
       '19:45', '08:50', '15:40', '06:05', '15:00', '13:55', '05:55',
       '13:20', '05:05', '06:25', '17:30', '08:20', '19:55', '06:30',
       '14:05', '02:00', '09:40', '08:25', '20:25', '13:15', '02:15',
       '16:55', '20:45', '05:15', '19:50', '20:00', '06:10', '19:30',
       '04:45', '12:55', '18:15', '17:20', '15:25', '23:00', '12:00',
       '14:45', '11:50', '11:30', '14:40', '19:10', '06:00', '23:30',
       '07:35', '13:05', '12:30', '15:10', '12:50', '18:25', '16:30',
       '00:40', '06:50', '13:00', '19:15', '01:30', '17:00', '10:00',
       '19:35', '15:

In [169]:
df['Arrival_Time'].unique()

array(['01:10 22 Mar', '13:15', '04:25 10 Jun', ..., '06:50 10 Mar',
       '00:05 19 Mar', '21:20 13 Mar'], dtype=object)

I will perform some feature engineering and convert these two columns to categories.

In [170]:
df['Dep_Time'] = df['Dep_Time'].str.replace(":",".").astype(float)

In [171]:
df['Arrival_Time'] = df['Arrival_Time'].apply(lambda x : x.replace(":",".").split(" ")[0]).astype(float)

In [172]:
def categorize_time(hour):
    if 0 <= hour < 6:
        return 'Early Morning'
    elif 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

In [173]:
df['Dep_Time_Category'] = df['Dep_Time'].apply(categorize_time)
df['Arrival_Time_Category'] = df['Arrival_Time'].apply(categorize_time)

In [174]:
df.drop(['Dep_Time','Arrival_Time'],axis=1,inplace=True)

In [175]:
df.duplicated().sum()

312

In [176]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)

In [177]:
df

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Day_of_Journey,Month_of_Journey,Dep_Time_Category,Arrival_Time_Category
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897,Sunday,March,Night,Early Morning
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662,Wednesday,May,Early Morning,Afternoon
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882,Sunday,June,Morning,Early Morning
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1 stop,No info,6218,Sunday,May,Evening,Night
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1 stop,No info,13302,Friday,March,Afternoon,Night
...,...,...,...,...,...,...,...,...,...,...,...,...
9565,SpiceJet,Banglore,Delhi,BLR → DEL,2h 40m,non-stop,No check-in baggage included,3257,Tuesday,May,Early Morning,Morning
9566,Air Asia,Kolkata,Banglore,CCU → BLR,2h 30m,non-stop,No info,4107,Tuesday,April,Evening,Night
9567,Air India,Kolkata,Banglore,CCU → BLR,2h 35m,non-stop,No info,4145,Saturday,April,Evening,Night
9568,Vistara,Banglore,New Delhi,BLR → DEL,2h 40m,non-stop,No info,12648,Friday,March,Morning,Afternoon


In [178]:
df["Duration"]

0       2h 50m
1       7h 25m
2          19h
3       5h 25m
4       4h 45m
         ...  
9565    2h 40m
9566    2h 30m
9567    2h 35m
9568    2h 40m
9569    8h 20m
Name: Duration, Length: 9570, dtype: object

I will perform some feature engineering and convert this column to time in minutes.

In [179]:
def convert_to_minutes(time):
    if 'h' in time:
        parts = time.split()
        hours = int(parts[0][:-1]) * 60
        if len(parts) > 1:
            minutes = int(parts[1][:-1])
        else:
            minutes = 0
        return hours + minutes
    else:
        return int(time[:-1]) * 60

In [180]:
df['Duration_minutes'] = df['Duration'].apply(convert_to_minutes)

In [181]:
df.drop(['Duration','Route'],axis=1,inplace=True)

In [182]:
df['Total_Stops'].value_counts()

1 stop      5362
non-stop    2925
2 stops     1242
3 stops       40
4 stops        1
Name: Total_Stops, dtype: int64

I will drop "4 stops" because it appears only once in the DataFrame.

In [183]:
df.drop(df[df["Total_Stops"]=='4 stops'].index, inplace=True)

In [184]:
df['Additional_Info'].value_counts()

No info                         7506
In-flight meal not included     1756
No check-in baggage included     272
1 Long layover                    19
Change airports                    6
Business class                     4
No Info                            3
1 Short layover                    1
Red-eye flight                     1
2 Long layover                     1
Name: Additional_Info, dtype: int64

I will drop ["1 Short layover","Red-eye flight","2 Long layover"] because it appears only once in the DataFrame, and replace "No Info" with "No info"

In [185]:
df['Additional_Info'].replace({'No Info': 'No info'},inplace=True)

In [186]:
threshold = 2
df['Additional_Info'].where(df['Additional_Info'].map(df['Additional_Info'].value_counts()) >= threshold, np.nan, inplace=True)

In [187]:
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

In [188]:
df['Price'].describe()

count     9566.000000
mean      9197.151474
std       4664.155400
min       1759.000000
25%       5583.000000
50%       8475.500000
75%      12395.000000
max      79512.000000
Name: Price, dtype: float64

In [189]:
px.box(data_frame=df,x='Price')

In [190]:
df

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Day_of_Journey,Month_of_Journey,Dep_Time_Category,Arrival_Time_Category,Duration_minutes
0,IndiGo,Banglore,New Delhi,non-stop,No info,3897,Sunday,March,Night,Early Morning,170
1,Air India,Kolkata,Banglore,2 stops,No info,7662,Wednesday,May,Early Morning,Afternoon,445
2,Jet Airways,Delhi,Cochin,2 stops,No info,13882,Sunday,June,Morning,Early Morning,1140
3,IndiGo,Kolkata,Banglore,1 stop,No info,6218,Sunday,May,Evening,Night,325
4,IndiGo,Banglore,New Delhi,1 stop,No info,13302,Friday,March,Afternoon,Night,285
...,...,...,...,...,...,...,...,...,...,...,...
9561,SpiceJet,Banglore,Delhi,non-stop,No check-in baggage included,3257,Tuesday,May,Early Morning,Morning,160
9562,Air Asia,Kolkata,Banglore,non-stop,No info,4107,Tuesday,April,Evening,Night,150
9563,Air India,Kolkata,Banglore,non-stop,No info,4145,Saturday,April,Evening,Night,155
9564,Vistara,Banglore,New Delhi,non-stop,No info,12648,Friday,March,Morning,Afternoon,160


What is the average ticket price for each airline?

In [191]:
avg_price_by_airline = df.groupby('Airline')['Price'].mean().reset_index()

In [192]:
px.bar(avg_price_by_airline, x='Airline', y='Price', title='Average Ticket Price by Airline')

How does the price vary based on the total number of stops?

In [193]:
price_by_stops = df.groupby('Total_Stops')['Price'].mean().reset_index()

In [194]:
 px.bar(price_by_stops, x='Total_Stops', y='Price', title='Price Variation Based on Total Stops')

Is there a correlation between the month of the journey and ticket prices?

In [195]:
px.scatter(df, x='Month_of_Journey', y='Price', title='Correlation Between Month of Journey and Ticket Prices')

Which airline has the most flights ?

In [196]:
df['Airline'].value_counts().reset_index()

,index,Airline
0,Jet Airways,3388
1,IndiGo,1797
2,Air India,1610
3,Multiple carriers,1123
4,SpiceJet,725
5,Vistara,414
6,Air Asia,302
7,GoAir,185
8,Multiple carriers Premium economy,13
9,Jet Airways Business,6


Are there specific days of the week when ticket prices are generally higher or lower?

In [197]:
px.scatter(df, x='Day_of_Journey', y='Price',size='Price')

Is there a correlation between the number of stops and the duration of the flight?

In [198]:
px.scatter(df, x='Total_Stops', y='Duration_minutes',title='Correlation between Number of Stops and Flight Duration',size='Price')


What are the most common source and destination combinations?

In [199]:
df.groupby(['Source', 'Destination']).size().reset_index(name='Count')

,Source,Destination,Count
0,Banglore,Delhi,1000
1,Banglore,New Delhi,892
2,Chennai,Kolkata,311
3,Delhi,Cochin,4065
4,Kolkata,Banglore,2692
5,Mumbai,Hyderabad,606


In [200]:
x = df.drop("Price" , axis =1 )
y = df['Price']

In [201]:
models = list()
models.append(("LR" , LinearRegression()))
models.append(("DT" , DecisionTreeRegressor()))
models.append(("KNN" , KNeighborsRegressor()))
models.append(("RF" , RandomForestRegressor()))
models.append(("XGB" , XGBRegressor()))

In [202]:
scorer = make_scorer(r2_score)

In [203]:
Encoder = ColumnTransformer(transformers=[("BE" , BinaryEncoder() , ['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info',
       'Day_of_Journey', 'Month_of_Journey', 'Dep_Time_Category',
       'Arrival_Time_Category'])] , remainder='passthrough')

In [204]:
for model in models:
    steps = []
    steps.append(("Encoder" , Encoder))
    steps.append(("Scaler" , StandardScaler()))
    steps.append(model)
    pipeline = Pipeline(steps = steps)
    res = cross_validate(pipeline,x,y,cv = 5 , scoring = scorer ,error_score='raise', return_train_score=True)
    print(f"Model {model[0]} train r2_score is {res['train_score'].mean()}")
    print(f"Model {model[0]} test r2_score is {res['test_score'].mean()}")    
    print("*" * 100)

Model LR train r2_score is 0.5731075221130982
Model LR test r2_score is 0.5709367002455217
****************************************************************************************************
Model DT train r2_score is 0.9740476010148249
Model DT test r2_score is 0.656789019264538
****************************************************************************************************
Model KNN train r2_score is 0.8065180462442596
Model KNN test r2_score is 0.6988655774702142
****************************************************************************************************
Model RF train r2_score is 0.9515921184939546
Model RF test r2_score is 0.7851747581800919
****************************************************************************************************
Model XGB train r2_score is 0.9213016111470009
Model XGB test r2_score is 0.8162836521181875
****************************************************************************************************


### hyperparameter tuning for XGBRegressor algorithm

In [205]:
#steps = []
#steps.append(("Encoder" , Encoder))
#steps.append(("Scaler" , StandardScaler()))
#steps.append(("XGB" , XGBRegressor()))
#pipeline = Pipeline(steps = steps)

In [206]:
#params = {
    #'XGB__n_estimators': [75,65,70],
    #'XGB__max_depth': [8,6, 7],
#}


In [207]:
#grid = GridSearchCV(estimator = pipeline , param_grid = params , cv = 5  ,return_train_score=True ,  scoring = scorer )

In [208]:
#grid.fit(x,y)

In [209]:
#grid.best_params_

In [210]:
#grid.cv_results_["mean_train_score"].max()

In [211]:
#grid.cv_results_["mean_test_score"].max()

### MODELING WITH XGBClassifier

In [212]:
steps = []
steps.append(("Encoder" , Encoder))
steps.append(("Scaler" , StandardScaler()))
steps.append(("XGB" , XGBRegressor(max_depth = 7 ,n_estimators = 70 )))
pipeline = Pipeline(steps = steps)

In [213]:
pipeline.fit(x,y)

Pipeline(steps=[('Encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('BE', BinaryEncoder(),
                                                  ['Airline', 'Source',
                                                   'Destination', 'Total_Stops',
                                                   'Additional_Info',
                                                   'Day_of_Journey',
                                                   'Month_of_Journey',
                                                   'Dep_Time_Category',
                                                   'Arrival_Time_Category'])])),
                ('Scaler', StandardScaler()),
                ('XGB',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              co...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=7, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=70,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

### Deployment with stremlit

In [214]:
import joblib

In [215]:
joblib.dump(pipeline,"Model.pkl")

['Model.pkl']

In [216]:
joblib.dump(x.columns , "Inputs.pkl")

['Inputs.pkl']

In [217]:
x.columns

Index(['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info',
       'Day_of_Journey', 'Month_of_Journey', 'Dep_Time_Category',
       'Arrival_Time_Category', 'Duration_minutes'],
      dtype='object')

In [4]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import sklearn
Model = joblib.load("Model.pkl")
Inputs = joblib.load("Inputs.pkl")

def Make_Prdiction(Airline, Source, Destination, Total_Stops, Additional_Info,
       Day_of_Journey, Month_of_Journey, Dep_Time_Category,
       Arrival_Time_Category, Duration_minutes):
    
    df = pd.DataFrame(columns=Inputs)
    df.at[0,"Airline"] = Airline
    df.at[0,"Source"] = Source
    df.at[0,"Destination"] = Destination
    df.at[0,"Total_Stops"] = Total_Stops
    df.at[0,"Additional_Info"] = Additional_Info
    df.at[0,"Day_of_Journey"] = Day_of_Journey
    df.at[0,"Month_of_Journey"] = Month_of_Journey
    df.at[0,"Dep_Time_Category"] = Dep_Time_Category
    df.at[0,"Arrival_Time_Category"] = Arrival_Time_Category
    df.at[0,"Duration_minutes"] = Duration_minutes
    result = Model.predict(df)
    return result[0]
def main():
    st.title("Flight Price")
    Airline= st.selectbox("Airline",['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet','Multiple carriers', 'GoAir', 'Vistara', 'Air Asia','Vistara Premium economy', 'Jet Airways Business','Multiple carriers Premium economy']) 
    Source = st.selectbox("Source",['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'])
    Destination = st.selectbox("Destination" ,['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'] )
    Total_Stops = st.selectbox("Number of Stops" ,['non-stop', '1 stops', '2 stop', '3 stops'])
    Additional_Info = st.selectbox("Additional Info" , ['No info', 'In-flight meal not included','No check-in baggage included', '1 Long layover','Change airports', 'Business class'])
    Day_of_Journey = st.selectbox("Day of Journey" ,['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'] )
    Month_of_Journey = st.selectbox("Month of Journey",['March', 'May', 'June', 'April'])
    Dep_Time_Category = st.selectbox("Departure Time",['Night', 'Early Morning', 'Morning', 'Evening', 'Afternoon'])
    Arrival_Time_Category = st.selectbox("Arrival Time",['Early Morning', 'Afternoon', 'Night', 'Morning', 'Evening'])
    Duration_minutes = st.slider("Duration from Departure to Arrival(minutes)" ,  min_value=60, max_value=2900, value=0, step=1)
    if st.button("Predict"):
        Results = Make_Prdiction(Airline, Source, Destination, Total_Stops, Additional_Info,Day_of_Journey, Month_of_Journey, Dep_Time_Category,Arrival_Time_Category, Duration_minutes)
        st.text(f"The total Price is : {Results.astype(int)}")
main()

Overwriting app.py
